<a href="https://colab.research.google.com/github/esshka/41-socks/blob/gh-pages/neat4full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Steps to Implement NEAT for XOR:
**Encoding Neural Networks:**

Define how neural networks will be encoded as genomes for evolution.
Include genes for nodes and connections, with each gene having attributes like weights, and maybe a flag indicating whether it's enabled or disabled.

**Initialization:**

Create an initial population of neural networks with simple topologies (e.g., no hidden nodes, random weights).

**Evaluation:**

Define a fitness function to evaluate how well each network performs on the XOR problem.
The fitness could be calculated as the inverse of the mean squared error over the four possible input pairs for XOR.

**Speciation:**

Implement a method to divide the population into species based on genomic similarity to ensure genetic diversity.
Define a compatibility distance to measure similarity/dissimilarity between genomes.

**Reproduction:**

Implement genetic operators like mutation (adding nodes and connections, altering weights) and crossover to produce offspring from parent networks.
Select parents based on fitness, possibly using a method like tournament selection.

**Next Generation:**

Select networks to form the next generation, considering both fitness and species diversity.
Determine how many networks from each species are allowed to reproduce and how many are carried to the next generation.

**Termination Condition:**

Define when to stop the evolution, e.g., when a solution is found, after a fixed number of generations, or when the fitness plateaus.

# Step 1: Encoding Neural Networks
Node Gene: Contains information about each node in the network (e.g., type of node: input, hidden, or output).
Connection Gene: Contains information about the connections between nodes (e.g., in-node, out-node, weight, and whether the connection is enabled).
Genome: A collection of node genes and connection genes that fully describes a neural network.

In [2]:
# Step 1: Encoding Neural Networks

import random

# Define Node Gene
class NodeGene:
    def __init__(self, node_id, node_type):
        """
        node_id: Unique identifier for the node.
        node_type: Type of node, can be 'input', 'hidden', or 'output'.
        """
        self.node_id = node_id
        self.node_type = node_type

# Define Connection Gene
class ConnectionGene:
    def __init__(self, in_node, out_node, weight, enabled, innovation):
        """
        in_node: Input node ID.
        out_node: Output node ID.
        weight: Weight of the connection.
        enabled: Boolean indicating whether the connection is enabled.
        innovation: Unique identifier to keep track of historical markings.
        """
        self.in_node = in_node
        self.out_node = out_node
        self.weight = weight
        self.enabled = enabled
        self.innovation = innovation

# Define Genome
class Genome:
    def __init__(self):
        self.nodes = {}  # Node genes: {node_id: NodeGene}
        self.connections = {}  # Connection genes: {innovation: ConnectionGene}

    def add_node_gene(self, node_gene):
        self.nodes[node_gene.node_id] = node_gene

    def add_connection_gene(self, connection_gene):
        self.connections[connection_gene.innovation] = connection_gene

    def randomize_weights(self, low=-1.0, high=1.0):
        for connection in self.connections.values():
            connection.weight = random.uniform(low, high)

# Example Usage
# Create a simple genome with 2 input nodes, 1 output node, and connections between them.
genome_example = Genome()

# Add node genes
for i in range(2):
    genome_example.add_node_gene(NodeGene(node_id=i, node_type='input'))
genome_example.add_node_gene(NodeGene(node_id=2, node_type='output'))

# Add connection genes with random weights
innovation_counter = 1  # To assign unique innovation numbers
for i in range(2):
    genome_example.add_connection_gene(
        ConnectionGene(
            in_node=i,
            out_node=2,
            weight=random.uniform(-1, 1),
            enabled=True,
            innovation=innovation_counter)
    )
    innovation_counter += 1

# Check the created genome
(genome_example.nodes, genome_example.connections)


({0: <__main__.NodeGene at 0x7bf56f284430>,
  1: <__main__.NodeGene at 0x7bf56f2841f0>,
  2: <__main__.NodeGene at 0x7bf56f284760>},
 {1: <__main__.ConnectionGene at 0x7bf56f2851e0>,
  2: <__main__.ConnectionGene at 0x7bf56f285240>})

# Step 2: Initialization
In this step, we'll create an initial population of neural networks. Each individual in the population will be represented by a Genome instance with a specific structure suitable for the XOR problem. The XOR problem involves 2 input values and 1 output value, so we'll set up our genomes accordingly.

To keep things simple for this initial population:

Each genome will have 2 input nodes and 1 output node.
Every input node will be connected to the output node.
The connection weights will be initialized with random values.

In [3]:
# Step 2: Initialization

class Population:
    def __init__(self, size):
        self.size = size
        self.genomes = [self._create_genome() for _ in range(size)]

    def _create_genome(self):
        genome = Genome()

        # Add node genes: 2 input nodes and 1 output node
        for i in range(2):
            genome.add_node_gene(NodeGene(node_id=i, node_type='input'))
        genome.add_node_gene(NodeGene(node_id=2, node_type='output'))

        # Add connection genes with random weights
        innovation_counter = 1
        for i in range(2):
            genome.add_connection_gene(
                ConnectionGene(
                    in_node=i,
                    out_node=2,
                    weight=random.uniform(-1, 1),
                    enabled=True,
                    innovation=innovation_counter)
            )
            innovation_counter += 1

        return genome

# Create an initial population of 100 genomes
pop_size = 100
population = Population(size=pop_size)

# Check the nodes and connections of the first genome in the population as an example
(population.genomes[0].nodes, population.genomes[0].connections)


({0: <__main__.NodeGene at 0x7bf56f284a00>,
  1: <__main__.NodeGene at 0x7bf56f284ac0>,
  2: <__main__.NodeGene at 0x7bf56f284b20>},
 {1: <__main__.ConnectionGene at 0x7bf56f284a90>,
  2: <__main__.ConnectionGene at 0x7bf56f2849d0>})

# Step 3: Evaluation
In this step, we will:

Define the XOR function.
Implement a method to compute the output of a neural network given its genome and an input.
Implement a fitness function to evaluate each genome in the population based on how well it approximates the XOR function.
 We'll calculate the mean squared error (MSE) between the neural network's output and the expected output for each pair, and use this error to derive a fitness score.

**Substeps:**

a. Neural Network Output:

Given a genome and an input, compute the output of the corresponding neural network.

b. Fitness Calculation:

Calculate the MSE between the expected and actual outputs for each input pair.
Define fitness as the inverse of the error (lower error -> higher fitness).

In [5]:
import numpy as np

def sigmoid(x):
    """Sigmoid activation function."""
    return 1 / (1 + np.exp(-x))

# Step 3a: Compute Neural Network Output

def activate_network(genome, inputs):
    """
    Given a genome and inputs, compute the output of the neural network.
    Note: This simple activation assumes no hidden nodes and direct connections
    from inputs to output.
    """
    output_node_id = 2  # Output node has ID=2 as per our initialization
    output_value = 0

    # Sum up the weighted inputs for the output node
    for conn in genome.connections.values():
        if conn.enabled:
            output_value += inputs[conn.in_node] * conn.weight

    return [sigmoid(output_value)]

# Example Usage: Activate the network for each XOR input
xor_inputs = [[0, 0], [0, 1], [1, 0], [1, 1]]
example_genome = population.genomes[0]  # Using the first genome in our population as an example

# Get network outputs for each input pair
network_outputs = [activate_network(example_genome, inp) for inp in xor_inputs]

# Check the network outputs for each XOR input
(network_outputs, xor_inputs)


([[0.5], [0.7305147551872365], [0.5667062774266426], [0.7799990934193402]],
 [[0, 0], [0, 1], [1, 0], [1, 1]])

In [6]:
# Step 3b: Fitness Calculation

# Expected outputs for XOR
xor_outputs = [[0], [1], [1], [0]]

def calculate_mse(output, target):
    """Calculate Mean Squared Error between output and target."""
    return np.mean((np.array(output) - np.array(target))**2)

def calculate_fitness(genome):
    """
    Calculate the fitness of a genome based on how well it approximates the XOR function.
    The fitness is calculated as the inverse of the mean squared error across all XOR inputs.
    """
    # Get network outputs for each input pair using sigmoid activation
    network_outputs = [activate_network(genome, inp) for inp in xor_inputs]

    # Calculate the mean squared error for each input-output pair
    mse = np.mean([calculate_mse(out, target) for out, target in zip(network_outputs, xor_outputs)])

    # Inverse of error as fitness (lower error -> higher fitness)
    return 1 / mse

# Calculate fitness for each genome in the population
fitness_values = [calculate_fitness(genome) for genome in population.genomes]

# Check the fitness values for the first few genomes in the population
fitness_values[:5]


[3.5753731883196167,
 3.5942301744050074,
 3.8945284400591333,
 3.9827194501077594,
 3.703837155602656]

# Step 4: Speciation

In this step, we will divide our population into species. The objective is to protect innovative solutions that are not yet fully optimized. By grouping similar genomes into species, we ensure that they primarily compete with similar solutions, rather than with the entire population.

**Key Points for Speciation:**

**Compatibility Distance:** A metric that measures the dissimilarity between two genomes. Genomes are typically compared based on the number and weights of mismatched genes.

**Species Threshold:** A threshold value for the compatibility distance to determine whether two genomes belong to the same species.

**Representative:** A genome chosen to represent the species. All other genomes are compared to this representative to decide if they belong to the species.
Adjusting Fitness: Fitness scores are often adjusted based on the number of genomes in the species to prevent larger species from taking over the population.

**Steps to Implement Speciation:**

Calculate Distance: Define a function to calculate the compatibility distance between two genomes.

Assign Species: Based on compatibility distances and a threshold, assign genomes to species.

Adjust Fitness: Optionally, adjust the fitness of genomes based on species size.


# 4.1: Calculate Distance

In [8]:
# Step 4.1: Calculate Compatibility Distance

# Coefficients for calculating genetic distance
c1 = 1.0  # Excess genes coefficient
c2 = 1.0  # Disjoint genes coefficient
c3 = 0.4  # Average weight difference coefficient

def calculate_distance(genome1, genome2):
    """
    Calculate the compatibility distance between two genomes.
    """
    # Get lists of connection genes
    conn_genes1 = list(genome1.connections.values())
    conn_genes2 = list(genome2.connections.values())

    # Sorting genes by innovation number for comparison
    conn_genes1.sort(key=lambda gene: gene.innovation)
    conn_genes2.sort(key=lambda gene: gene.innovation)

    # Indices for iterating through genes
    i = j = 0

    # Counters for genetic distance
    excess_genes = 0
    disjoint_genes = 0
    matching_genes = 0
    weight_diff_sum = 0

    # Iterate through genes to count excess, disjoint and find weight differences
    while i < len(conn_genes1) and j < len(conn_genes2):
        innov1 = conn_genes1[i].innovation
        innov2 = conn_genes2[j].innovation

        # Matching genes: compute weight differences
        if innov1 == innov2:
            weight_diff_sum += abs(conn_genes1[i].weight - conn_genes2[j].weight)
            matching_genes += 1
            i += 1
            j += 1
        # Disjoint genes: gene present in one genome but not the other
        elif innov1 < innov2:
            disjoint_genes += 1
            i += 1
        else:
            disjoint_genes += 1
            j += 1

    # Count genes from more advanced genome as excess
    excess_genes = len(conn_genes1[i:]) + len(conn_genes2[j:])

    # Average weight difference
    avg_weight_diff = weight_diff_sum / matching_genes if matching_genes > 0 else 0

    # Normalize for genome size (use max to avoid dividing by zero)
    N = max(len(conn_genes1), len(conn_genes2))
    N = 1 if N < 20 else N  # Normalization not needed for small genomes

    # Compatibility distance
    d = c1 * excess_genes/N + c2 * disjoint_genes/N + c3 * avg_weight_diff

    return d

# Example Usage: Calculate distance between the first two genomes in the population
distance_example = calculate_distance(population.genomes[0], population.genomes[1])

# Check the calculated compatibility distance
distance_example


0.1680577570228392

# 4.2: Assign Species
We'll assign each genome to a species based on the compatibility distance.
If a genome's distance to a species representative is below a specified threshold, it is assigned to that species.
If it doesn’t closely match any existing species, a new species is created.

In [19]:
# Step 4.2: Assign Species

# Threshold for compatibility distance to assign genomes to the same species
species_threshold = 1.0

class Species:
    def __init__(self, representative):
        self.representative = representative  # Genome representing the species
        self.genomes = [representative]  # Genomes assigned to the species
        self.age = 0  # Age of the species (in generations)

    def add_genome(self, genome):
        self.genomes.append(genome)

def assign_species(population):
    """
    Assign each genome in the population to a species.
    """
    species_list = []

    # Assign each genome to a species
    for genome in population.genomes:
        # Check compatibility with existing species
        assigned_to_species = False
        for species in species_list:
            distance = calculate_distance(genome, species.representative)
            # print(distance)
            if distance < species_threshold:
                species.add_genome(genome)
                assigned_to_species = True
                break

        # If not compatible with any existing species, create a new species
        if not assigned_to_species:
            species_list.append(Species(representative=genome))

    return species_list

# Assign genomes in the population to species
species_list = assign_species(population)
# population

# species_list[0].genomes

# Check the number of species and the number of genomes in the first few species
(num_species := len(species_list), [len(species.genomes) for species in species_list[:5]])


(1, [100])

# Step 5: Reproduction
In this step, we generate the next generation of genomes (neural networks) through reproduction. Reproduction involves selecting genomes to serve as parents and applying genetic operators to create offspring for the next generation.

Substeps:

**5.1 Selection:**
Choose genomes to reproduce based on their fitness. Genomes with higher fitness should be more likely to be selected.

**5.2 Crossover:**
Combine the genes of two parent genomes to create a child genome. Ensure that the resulting child genome inherits genes in a way that respects the historical markings (innovation numbers).

**5.3 Mutation:**
Introduce small changes to some genomes, such as altering connection weights, adding new connections, or adding new nodes.

**Reproduction Strategy:**

Elitism: Allow some of the fittest genomes to pass to the next generation without changes.

Crossover: Select pairs of genomes to create offspring through crossover.
Mutation: Apply mutations to create variations.

In [20]:
# Step 5.1: Selection

def tournament_selection(population, tournament_size=3):
    """
    Select a parent genome using tournament selection.
    Randomly select `tournament_size` genomes and return the fittest among them.
    """
    # Randomly select genomes for the tournament
    tournament_contestants = random.sample(population.genomes, tournament_size)

    # Select the fittest genome among the contestants
    fittest_genome = max(tournament_contestants, key=calculate_fitness)

    return fittest_genome

# Example Usage: Select a parent genome using tournament selection
selected_parent = tournament_selection(population)

# Check the fitness of the selected parent
calculate_fitness(selected_parent)


3.8945284400591333

In [ ]:
# Step 5.2: Crossover

def crossover(parent1, parent2):
    """
    Produce a child genome through crossover from two parent genomes.
    """
    # Ensure parent1 is the more fit parent
    if calculate_fitness(parent2) > calculate_fitness(parent1):
        parent1, parent2 = parent2, parent1

    # Get lists of connection genes
    conn_genes1 = list(parent1.connections.values())
    conn_genes2 = list(parent2.connections.values())

    # Create a lookup table for faster search in parent2 genes
    conn_genes2_lookup = {gene.innovation: gene for gene in conn_genes2}

    # Create a child genome
    child_genome = Genome()

    # Inherit genes from parents
    for gene1 in conn_genes1:
        # If the gene is present in both parents, inherit from either parent randomly
        if gene1.innovation in conn_genes2_lookup:
            gene2 = conn_genes2_lookup[gene1.innovation]
            inherited_gene = random.choice([gene1, gene2])
        # If the gene is only present in the more fit parent, inherit it
        else:
            inherited_gene = gene1

        # Add the inherited gene to the child genome
        child_genome.add_connection_gene(ConnectionGene(
            in_node=inherited_gene.in_node,
            out_node=inherited_gene.out_node,
            weight=inherited_gene.weight,
            enabled=inherited_gene.enabled,
            innovation=inherited_gene.innovation
        ))

    # Inherit node genes from parent1
    for node_id, node_gene in parent1.nodes.items():
        child_genome.add_node_gene(NodeGene(node_id=node_id, node_type=node_gene.node_type))

    return child_genome

# Example Usage: Produce a child genome through crossover from two parent genomes
parent1 = tournament_selection(population)
parent2 = tournament_selection(population)

child_genome = crossover(parent1, parent2)

# Check the connections in the child genome
child_genome.connections
